In [1]:
%reload_kedro
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder


                    INFO     Resolved project path as:                                              ]8;id=787120;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=602195;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#164\164]8;;\
                             C:\Users\yair0\OneDrive\Escritorio\Projectbreitong.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/14/24 20:32:03] INFO     Kedro project breitong                                                 ]8;id=679311;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=116621;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=171733;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=62519;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=300538;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=757611;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

In [2]:
df_champs = catalog.load("champions")
df_grades_2023 = catalog.load("grades_2023")
df_questionnaires = catalog.load("questionnaire_sense")
df_leadership = catalog.load("leadership_sense")
df_primary_psycho = catalog.load("primary_psycho")
df_secondary_psycho = catalog.load("secondary_psycho")


[06/14/24 20:32:24] INFO     Loading data from champions (ExcelDataset)...                      ]8;id=698216;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=440154;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

                    INFO     Loading data from grades_2023 (ExcelDataset)...                    ]8;id=567740;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=938785;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

[06/14/24 20:32:25] INFO     Loading data from questionnaire_sense (CSVDataset)...              ]8;id=70708;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=933116;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

                    INFO     Loading data from leadership_sense (CSVDataset)...                 ]8;id=399966;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=633466;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

                    INFO     Loading data from primary_psycho (ExcelDataset)...                 ]8;id=327181;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=476463;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

[06/14/24 20:32:26] INFO     Loading data from secondary_psycho (ExcelDataset)...               ]8;id=604310;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=966651;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

In [3]:
def remove_high_null_columns(df: pd.DataFrame, threshold: float = 0.5) -> pd.DataFrame:
    """
    Remove columns from a DataFrame that have more than a specified threshold of null values.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    threshold (float): The threshold for the percentage of null values. Columns with a percentage of null values greater than this threshold will be removed. Default is 0.5 (50%).

    Returns:
    pd.DataFrame: The DataFrame with columns removed that had more than the specified threshold of null values.
    """
    # Calculate the percentage of null values for each column
    null_percentage = df.isnull().mean()
    # Filter out columns that have more than the specified threshold of null values
    columns_to_keep = null_percentage[null_percentage <= threshold].index
    # Return the DataFrame with the selected columns
    return df[columns_to_keep]

In [4]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the dataframe by applying text corrections to both column headers 
    and string type columns. Additionally, remove columns with a high percentage of null values.
    
    This function will:
    - Convert column headers to lowercase.
    - Strip leading and trailing whitespaces from column headers.
    - Apply lowercase and strip leading/trailing whitespaces to all string columns.
    - Remove columns with a high percentage of null values using the predefined `remove_high_null_columns` function.
    
    Parameters:
    df (pd.DataFrame): The input dataframe to be cleaned.
    
    Returns:
    pd.DataFrame: The cleaned dataframe.
    """
    df = df.copy()
    # Clean column headers
    columns = df.columns.tolist()
    df.columns = [x.lower().strip().replace(".1", "") for x in columns]

    for col in df.select_dtypes(include=['object']).columns:
        df.loc[df[col]=="ERROR", col] = np.nan
    
    # Remove high null columns
    df = remove_high_null_columns(df)
    df = df.infer_objects()

    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.lower().str.strip()

    return df


In [5]:
df_champs_final = clean_dataframe(df_champs)
df_champs_final.drop(columns=['dni papá encriptado', 'dni mamá encriptado', 'salud'], inplace=True)

df_grades_2023_final = clean_dataframe(df_grades_2023)
df_questionnaires_final = clean_dataframe(df_questionnaires)
df_leadership_final = clean_dataframe(df_leadership)
df_leadership_final = df_leadership_final[["dni encriptado"] + df_leadership_final.iloc[:, -10:].columns.tolist()]
df_primary_psycho_final = df_primary_psycho[["DNI ENCRIPTADO"]+df_primary_psycho.iloc[:, 35:65].columns.to_list() + df_primary_psycho.iloc[:, -8:-2].columns.to_list()]
df_primary_psycho_final = clean_dataframe(df_primary_psycho_final)

df_secondary_psycho_final = clean_dataframe(df_secondary_psycho)
cols = df_secondary_psycho_final.iloc[:, -196:].columns.tolist()
for col in cols:
    label_encoder = LabelEncoder()
    df_secondary_psycho_final[col] = label_encoder.fit_transform(df_secondary_psycho_final[col])
df_secondary_psycho_final = df_secondary_psycho_final[["dni encriptado"] + cols]


In [6]:
df_final = df_champs_final.merge(df_grades_2023_final, on="dni encriptado", how="left")
df_final = df_final.merge(df_questionnaires_final, left_on="dni encriptado", right_on="dni del champ encriptado", how="left")
df_final = df_final.merge(df_leadership_final, on="dni encriptado", how="left")
df_final = df_final.merge(df_primary_psycho_final, on="dni encriptado", how="left")
df_final = df_final.merge(df_secondary_psycho_final, on="dni encriptado", how="left")
df_final.drop(columns=df_final.filter(like="_y").columns.to_list(), inplace=True)
df_final.rename(columns={col: col.replace("_x", "") for col in df_final.columns if "_x" in col}, inplace=True)
df_final = remove_high_null_columns(df_final, 0.6).dropna()
df_final

,dni encriptado,sexo,fecha nac.,edad,grado,sede,región,origen,estado,año de ingreso,modalidad de ingreso,cantera,matemática,comunicación,inglés,ciencias,dpsc,educación física,arte,conducta,promedio,prefinal,por curso,nra,dni del champ encriptado,fecha de nacimiento de champ,edad de champ,sede de champ,grado de champ,"champ tiene alguna alergia, condición o enfermedad",champ para talleres virtuales se puede conectar de,¿qué tema te interesa champ?,¿con quienes vives?\npor ejemplo: con mi padres y hermanos,¿cuáles son tus pasatiempos favoritos? por ejemplo: leer y jugar videojuegos,¿cuáles son tus cursos favoritos?\npor ejemplo: matemática y comunicación,¿cuál es tu deporte favorito?\npor ejemplo: fútbol,¿qué te gustaría estudiar o ser de grande?\npor ejemplo: doctor,¿qué te gustaría hacer para mejorar el perú?,¿te gustaría estudiar en una universidad extranjera?,"cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023","en perú champs nos preocupa tu bienestar. ¿hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? este espacio es confidencial y no afecta tu beca",¿tiene otro champ?,dni del apoderado encriptado,qué relación tiene con champ,su champ tiene un apoderada/o adicional a usted,¿tienes un negocio propio o emprendimiento?,la conexión a internet en su casa es,tu internet es,"¿algún familiar con el que viva tiene alguna condición, discapacidad o enfermedad crónica que requiera de cuidados constante?",cuéntanos brevemente cómo se encuentran actualmente,¿quién apoya a su champ con sus tareas?,champ para talleres virtuales se puede conectar de_sentiment,¿qué tema te interesa champ?_sentiment,¿con quienes vives?\npor ejemplo: con mi padres y hermanos_sentiment,¿cuáles son tus pasatiempos favoritos? por ejemplo: leer y jugar videojuegos_sentiment,¿cuáles son tus cursos favoritos?\npor ejemplo: matemática y comunicación_sentiment,¿cuál es tu deporte favorito?\npor ejemplo: fútbol_sentiment,¿qué te gustaría estudiar o ser de grande?\npor ejemplo: doctor_sentiment,¿qué te gustaría hacer para mejorar el perú?_sentiment,¿te gustaría estudiar en una universidad extranjera?_sentiment,"cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023_sentiment","en perú champs nos preocupa tu bienestar. ¿hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? este espacio es confidencial y no afecta tu beca_sentiment",¿tiene otro champ?_sentiment,dni del apoderado encriptado_sentiment,qué relación tiene con champ_sentiment,su champ tiene un apoderada/o adicional a usted_sentiment,¿tienes un negocio propio o emprendimiento?_sentiment,la conexión a internet en su casa es_sentiment,tu internet es_sentiment,"¿algún familiar con el que viva tiene alguna condición, discapacidad o enfermedad crónica que requiera de cuidados constante?_sentiment",cuéntanos brevemente cómo se encuentran actualmente_sentiment,¿quién apoya a su champ con sus tareas?_sentiment,1. preferiría seguir a alguien antes que ser el líder.,2. estoy bastante seguro(a) de saber quién soy y qué quiero en la vida.,3. yo no necesito tener amistades cercanas como otros chicos.,4. frecuentemente me desagrada hacer cosas que otros esperan de mí.,5. hago lo mejor que puedo para no herir los sentimientos de los demás.,6. puedo contar con que mis padres me entiendan.,7. algunas personas piensan que soy un poco presumido(a).,"8. pase lo que pase, nunca consumiría drogas.",9. siempre trato de hacer lo que es correcto.,10. me gusta cómo me veo.,"11. aunque me dan ataques incontrolables de hambre en los que como muchísimo, odio el peso que subo.",12. casi nada de lo que pasa parece hacerme sentir feliz o triste.,13. parece que tengo un problema para llevarme bien con otros adolescentes.,14. me siento muy avergonzado(a) contándole a la gente cómo fui

In [7]:
df_final.to_csv("data/01_raw/df_final.csv", index=False)

In [10]:
df_primary_psycho_final

,dni encriptado,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,intrapersonal,interpersonal,manejo del estrés,adaptabilidad,impresión positiva,tot. coc. emoc.
0,weiyyyds,3,2,1,4,2,2,2,3,2,3,3,2,3,3,4,3,2,3,3,2,3,3,3,3,3,2,2,4,2,3,98.0,102.0,83.0,101.0,93.0,119.0
1,ryiraaiz,4,4,3,4,3,4,2,4,4,4,2,1,3,4,4,4,4,4,4,3,4,4,4,3,2,1,4,2,3,1,114.0,102.0,115.0,117.0,95.0,146.0
2,rydzdeyr,4,2,3,3,2,2,2,3,3,3,2,4,3,2,4,3,2,3,2,4,2,2,3,2,2,2,2,2,4,2,96.0,93.0,92.0,93.0,100.0,117.0
3,ryssdwie,3,2,4,3,3,4,4,3,4,4,2,3,3,3,4,3,3,4,4,3,3,4,4,4,1,3,4,3,3,3,116.0,102.0,110.0,115.0,108.0,151.0
4,ryiiwinn,3,2,3,4,3,2,3,4,4,3,3,1,3,2,3,3,4,3,3,1,2,4,3,3,3,3,4,3,4,3,87.0,102.0,118.0,107.0,95.0,133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,wzyywwsz,3,2,4,4,4,2,4,4,4,3,2,4,4,2,4,4,4,3,4,3,2,3,4,4,2,3,4,4,4,4,106.0,112.0,131.0,117.0,118.0,159.0
762,weddzwye,3,1,2,3,4,2,2,4,4,2,3,3,2,2,3,2,4,3,2,2,2,3,4,2,1,3,4,4,4,3,97.0,103.0,131.0,83.0,85.0,126.0
763,rwzandzr,3,2,4,4,3,3,3,3,3,4,4,3,3,2,2,3,3,3,4,2,2,3,3,4,2,3,3,3,4,2,104.0,99.0,106.0,114.0,105.0,139.0
764,rwwednia,4,4,4,4,4,4,4,3,4,4,4,1,4,4,4,4,3,4,4,3,4,4,4,4,3,3,3,4,3,4,123.0,122.0,107.0,124.0,124.0,167.0
